Let's start with loading the training and test data:

In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
os.listdir()

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 '.Rapp.history',
 '.RDataTmp',
 '.Rhistory',
 'GBM.rds',
 'GBM.stack.rds',
 'Kaggle Mercari Price Suggestion Challenge IPython Kernel.ipynb',
 'KaggleMercariPriceSuggestionChallengeKernel.Rmd',
 'Mercari Price Suggestion Challenge | Kaggle.pdf',
 'merged_table.rds',
 'mini_subtrain.rds',
 'mini_subtrain_locked.rds',
 'mini_train.rds',
 'new_stack_set.rds',
 'new_stack_set2.rds',
 'RF.BC.rds',
 'RF.rds',
 'RF.stack.rds',
 'RF.stack2nd.rds',
 'sample_submission.csv',
 'sample_submission.csv.7z',
 'second.stack.set.rds',
 'subtrain.rds',
 'subtrain_locked.rds',
 'SVM.rds',
 'SVM.stack.rds',
 'test.csv',
 'test.rds',
 'test.tsv.7z',
 'test_locked.rds',
 'train.csv',
 'train.rds',
 'train.tsv.7z',
 'v.txt',
 'validation.rds',
 'validation_locked.rds',
 'xGBM.BC.rds',
 'xGBM.BC2.rds',
 'xGBM.rds',
 'xGBM.stack.rds',
 'xGBM.stack2nd.rds']

In [4]:
train = pd.read_csv("train.csv")

In [5]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


In [7]:
test = pd.read_csv("test.csv")

In [8]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
test_id              693359 non-null int64
name                 693359 non-null object
item_condition_id    693359 non-null int64
category_name        690301 non-null object
brand_name           397834 non-null object
shipping             693359 non-null int64
item_description     693359 non-null object
dtypes: int64(3), object(4)
memory usage: 37.0+ MB


First, let's remove the train_id from the training set, and map the numeric and character variables to being with:

In [10]:
train = train.drop("train_id",axis = 1)
train.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 7 columns):
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 79.2+ MB


In [59]:
Numeric_features_select = np.logical_or((pd.Series(train.dtypes)) == "int64",(pd.Series(train.dtypes) == "float64")).tolist()

In [67]:
Numeric_features = pd.Series(train.columns)[Numeric_features_select].tolist()

In [68]:
Numeric_features

['item_condition_id', 'price', 'shipping']